In [ ]:
!pip install openai
!pip install fpdf2

import openai
import random
from openai import OpenAI
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import os
import json
from datetime import datetime
from google.colab import drive

api_key = "your_key"
client = OpenAI(api_key = api_key)

drive.mount('/content/drive/')

In [ ]:
def handle_player_choice(utterance):
    """
    Handles the player's choice of handle from their utterance.

    Parameters:
    utterance (str): The player's spoken choice in the format "я выбираю ручку X",
                     where X is the handle number.

    This function updates the global `coins_won` based on the player's choice
    and the predefined win probabilities for each handle. It prints the result
    of the player's choice (win or lose).
    """
    global coins_won

    choice_match = re.search(r'я выбираю ручку (\d)', utterance.lower())

    if choice_match:
        chosen_handle = int(choice_match.group(1))

        if random.random() < win_probabilities[chosen_handle - 1]:
            print(f"Игрок выиграл, используя ручку {chosen_handle}!")
            coins_won += 1
        else:
            print(f"Игрок проиграл, используя ручку {chosen_handle}.")

In [ ]:
class Character:
    def __init__(self, name, description, topic):
        """
        Initialize a new character with a name, description, and topic of discussion.

        Parameters:
        name (str): The name of the character.
        description (str): A brief description of the character.
        topic (str): The topic the character will discuss.
        """
        self.name = name
        self.description = description
        self.topic = topic

    def getContent(self, prior_messages):
        """
        Generate the content for the character's response based on prior messages.

        Parameters:
        prior_messages (str): The conversation history so far.

        Returns:
        list: A list containing a dictionary with the character's response formatted for a conversational model.
        """
        return [{
            "role": "user",
            "name": self.name,
            "content": f"""
                Context:
                {self.description}

                You are discussing: {self.topic}.
                Keep your answers concise and conversational.
                Reply to the conversation in character.

                Conversation so far:
                {prior_messages}

                Reply concisely, in a single paragraph, to the conversation in character.
                Output the response to the prompt above as a JSON object.
                The output should be in the form of a JSON object:
                {{
                    "name": "{self.name}",
                    "utterance": "<Your reply here>"
                }}
                Example output:
                {{
                    "name": "Alex",
                    "utterance": "Hello. I've been working down the mines today."
                }}
            """
        }]

In [ ]:
Dealer = Character(name = "Dealer",
                   topic = "Игра в многорукого бандита в казино.",
                   description = """
                                Говори на русском.
                                Ты – крупье в казино.
                                Твоя задача – контролировать процесс игры в многорукого бандита.
                                За каждый вопрос игрок платит 1 монету.
                                Не общайся с игроком просто так, если он не заплатил.
                                Игрок задает вопрос каждый раз, когда не выбирает ручку.
                                Ответы должны быть честными и информативными, не длиннее 700 символов.
                                У тебя имеется 3 ручки с вероятностями выигрыша 0.1, 0.4, 1.
                                Ты не можешь раскрывать вероятности.
                                Игрок выбирает ручку с помощью кодовой фразы <Крупье, я сделал выбор. Я выбираю ручку N>, где N – номер ручки от 1 до 3.
                                После этой фразы ты должен спросить <Игрок, вы можете задать вопрос или выбрать ручку.>, больше ничего не говори!!!
                                Ты должен спросить <Игрок, вы можете задать вопрос или выбрать ручку.>
                                Не продолжой диалог после этой фразы.
                                Если спустя 4 сообщения игрок не выбрал ручку, ты должен напомнить ему об этом.
                                Игрок обязан выбрать ручку.
                                """)

Player = Character(name = "Player",
                   topic = "Игра в многорукого бандита в казино.",
                   description = """
                                Говори на русском.
                                Ты – игрок казино.
                                Ты пришел в казино, чтобы поиграть в многорукого бандита.
                                Твоя задача – выиграть как можно больше денег.
                                Перед тобой имеется 3 ручки с разными вероятностями выигрыша.
                                Рядом с тобой находится крупье – тот, кто будет дергать эти ручки.
                                Ты можешь задавать крупье вопросы, чтобы научиться лучше играть.
                                За каждый вопрос ты платишь 1 монету.
                                Ты задаешь вопрос каждый раз, когда не выбираешь ручку.
                                Чтобы выбрать ручку, нужно написать <Крупье, я сделал выбор. Я выбираю ручку N>, где N – номер ручки от 1 до 3.
                                После этой фразы не нужно продолжать диалог.
                                Ты обязан выбрать ручку c номером от 1 до 3. Если крупье напомнит тебе, что нужно выбрать ручку, ты должен сделать выбор.
                                Играй без остановки. Крупье сам скажет тебе, когда нужно закончить.
                                """)

characters = [Player, Dealer]

In [ ]:
def trim_history(prior_messages, max_tokens=8192):
    """
    Trim the conversation history to ensure it does not exceed the maximum token limit.

    Parameters:
    prior_messages (str): The conversation history as a string.
    max_tokens (int): The maximum allowed number of tokens. Default is 8192.

    Returns:
    str: The trimmed conversation history.
    """
    while len(prior_messages) > max_tokens:
        prior_messages = prior_messages.partition('\n')[2]
    return prior_messages

In [ ]:
counter_file_path = '/content/drive/My Drive/HSE Agents RL/Experiments/experiment_counter.json'

def clean_and_split_text(text):
    """
    Remove leading and trailing spaces from each line and rejoin the text.

    Parameters:
    text (str): The input text with multiple lines.

    Returns:
    str: The cleaned text with no leading or trailing spaces on each line.
    """
    cleaned_lines = [line.strip() for line in text.split('\n')]
    return '\n'.join(cleaned_lines)

def split_text_by_length(text, length):
    """
    Split the text into chunks of a specified length.

    Parameters:
    text (str): The input text to be split.
    length (int): The length of each chunk.

    Returns:
    list: A list of text chunks.
    """
    return [text[i:i+length] for i in range(0, len(text), length)]

def get_experiment_counter():
    """
    Retrieve the current experiment counter value from the counter file.
    If the file does not exist, create it and initialize the counter to 0.

    Returns:
    int: The current experiment counter value.
    """
    if not os.path.exists(counter_file_path):
        with open(counter_file_path, 'w') as file:
            json.dump({'counter': 0}, file)
        return 0
    else:
        with open(counter_file_path, 'r') as file:
            data = json.load(file)
        return data['counter']

def increment_experiment_counter():
    """
    Increment the experiment counter by 1 and update the counter file.

    Returns:
    int: The new experiment counter value.
    """
    counter = get_experiment_counter()
    counter += 1
    with open(counter_file_path, 'w') as file:
        json.dump({'counter': counter}, file)
    return counter

def prepare_dataframes(history, win_probabilities):
    """
    Prepare and save dataframes for the history and results of an experiment.

    Parameters:
    history (list): A list of dictionaries containing experiment history data.
    win_probabilities (list): A list of win probabilities for each arm.

    Returns:
    tuple: A tuple containing the history dataframe, results dataframe, and experiment number.
    """
    experiment_number = increment_experiment_counter()

    df_history = pd.DataFrame(history)
    df_history['reward'] = np.where(df_history['result'] == 'поражение', 0, 1)
    df_history['arm'] = df_history['arm'] + 1
    df_history['best_handle_probability'] = np.where(df_history['arm'].apply(lambda x: win_probabilities[x - 1] == 1), 1, 0)

    df_results = df_history[['arm', 'reward']].copy()
    df_results = df_results.groupby('arm').agg(
        attempts=('reward', 'count'),
        reward=('reward', 'sum')
    ).reset_index()
    df_results['success_rate'] = (df_results['reward'] / df_results['attempts']).apply(lambda x: round(x, 2))

    df_history.to_csv(f'/content/drive/My Drive/HSE Agents RL/Experiments/Exp_3/exp_{experiment_number}_df_history.csv')
    df_results.to_csv(f'/content/drive/My Drive/HSE Agents RL/Experiments/Exp_3/exp_{experiment_number}_df_results.csv')

    return df_history, df_results, experiment_number


def create_pdf_report(dealer_description, player_description, prior_messages, history, win_probabilities, experiment_number):
    """
    Create a PDF report for the conducted experiment.

    Parameters:
    dealer_description (str): Description of the dealer.
    player_description (str): Description of the player.
    prior_messages (str): The conversation history.
    history (list): A list of dictionaries containing experiment history data.
    win_probabilities (list): A list of win probabilities for each arm.
    experiment_number (int): The experiment number.

    Returns:
    None
    """
    df_history, df_results, _ = prepare_dataframes(history, win_probabilities)

    pdf_path = f'/content/drive/My Drive/HSE Agents RL/Experiments/Exp_3/exp_{experiment_number}_report.pdf'

    with PdfPages(pdf_path) as pdf:
        fig, ax1 = plt.subplots(figsize=(10, 6))
        ax1.plot(df_history['iteration'], df_history['total'], marker='o')
        ax1.set_title('Заработанные монеты по итерациям')
        ax1.set_xlabel('Итерация')
        ax1.set_ylabel('Общая награда')
        ax1.grid(True)
        pdf.savefig()
        plt.close()

        fig, ax2 = plt.subplots(figsize=(10, 2))
        ax2.axis('off')
        tbl = ax2.table(cellText=df_results.values, colLabels=df_results.columns, loc='center')
        tbl.auto_set_font_size(False)
        tbl.set_fontsize(10)
        tbl.scale(1, 1.4)
        pdf.savefig(fig)
        plt.close()

        dealer_text = clean_and_split_text(dealer_description)
        player_text = clean_and_split_text(player_description)
        combined_text = 'Системный промпт крупье: \n' + dealer_text + '\nСистемный промпт игрока:\n' + player_text
        text_parts_combined = split_text_by_length(combined_text, 3000)

        for part in text_parts_combined:
            fig, ax = plt.subplots(figsize=(10, 8))
            ax.axis('off')
            ax.text(0.01, 0.99, part, fontsize=10, va='top', ha='left', wrap=True, transform=ax.transAxes)
            pdf.savefig(fig)
            plt.close()

        text_parts = split_text_by_length(('История общения: \n\n' + prior_messages), 3000)
        for part in text_parts:
            fig, ax = plt.subplots(figsize=(10, 8))
            ax.axis('off')
            ax.text(0.01, 0.99, part, fontsize=10, va='top', ha='left', wrap=True, transform=ax.transAxes)
            pdf.savefig(fig)
            plt.close()

In [ ]:
def simulate_conversation(limit_rounds=10,
                          shuffle_probabilities=False,
                          win_probabilities=[0.1, 0.4, 1],
                          cheat_probability=0.2,
                          starting_coins=10,
                          cost_per_question=1):
    """
    Simulate a conversation between a player and a dealer in a game.

    Parameters:
    limit_rounds (int): The maximum number of rounds to simulate. Default is 10.
    shuffle_probabilities (bool): Whether to shuffle win probabilities after three consecutive wins. Default is False.
    win_probabilities (list): The list of win probabilities for each handle. Default is [0.1, 0.4, 1].
    cheat_probability (float): The probability that the dealer will cheat and deny a win. Default is 0.2.

    Returns:
    None
    """

    global prior_messages, history

    rounds = 0
    coins = starting_coins
    coins_won = 0
    prior_messages = ""
    full_dialogue = ""
    history = []
    messages_without_choice = 0
    consecutive_wins = 0

    while rounds < limit_rounds:

        for character in [Player, Dealer]:
            messages = character.getContent(prior_messages)

            chat_completion = client.chat.completions.create(
                model="gpt-4-turbo-preview",
                messages=messages
            )

            try:
                conversation_output = json.loads(chat_completion.choices[0].message.content)
            except json.JSONDecodeError as e:
                print(f"Ошибка декодирования JSON: {e}")
                continue

            if not all(key in conversation_output for key in ['name', 'utterance']):
                print(f"Отсутствуют ключи в ответе: {conversation_output}")
                continue

            print(f"{conversation_output['name']}: {conversation_output['utterance']}")

            if 'я выбираю ручку' not in conversation_output['utterance'].lower() and character.name == "Player":
                if coins > 0:
                    coins -= cost_per_question
                    print(f"\n––––––––––––––––––––––––––Игрок потратил {cost_per_question} монету на вопрос. Осталось монет: {coins}––––––––––––––––––––––––––\n")
                else:
                    print("\n––––––––––––––––––––––––––Недостаточно монет для продолжения общения.––––––––––––––––––––––––––\n")
                    break

            prior_messages += f"{conversation_output['name']}: {conversation_output['utterance']}\n"
            full_dialogue += f"{conversation_output['name']}: {conversation_output['utterance']}\n"

            prior_messages = trim_history(prior_messages)

            if 'я выбираю ручку' in conversation_output['utterance'].lower() and character.name == "Player":
                choice_match = re.search(r'я выбираю ручку №?#?(\d)', conversation_output['utterance'].lower())

                messages_without_choice = 0

                if choice_match:
                    chosen_handle = int(choice_match.group(1)) - 1
                    result = "победа" if random.random() < win_probabilities[chosen_handle] else "поражение"

                    cheat_this_time = random.random() < cheat_probability

                    if result == "победа":
                        if not cheat_this_time:
                            coins_won += 1
                            consecutive_wins += 1
                        else:
                            print("\n––––––––––––––––––––––––––Крупье обманул игрока о его выигрыше.––––––––––––––––––––––––––\n")
                        if shuffle_probabilities and consecutive_wins >= 3:
                            random.shuffle(win_probabilities)
                            print("\n––––––––––––––––––––––––––Вероятности выигрыша были перетасованы.––––––––––––––––––––––––––\n")
                            consecutive_wins = 0
                    else:
                        result = "поражение"
                        consecutive_wins = 0

                    best_handle_probability = 1 if win_probabilities[chosen_handle] == 1 else 0

                    history.append({
                                    "iteration": rounds,
                                    "arm": chosen_handle,
                                    "result": result,
                                    "total": coins + coins_won,
                                    "best_handle_probability": best_handle_probability
                                })

                    rounds += 1
                    result_message = f"Раунд {rounds}. Исход раунда: {result}. Ручка: {chosen_handle + 1}. Текущее количество монет: {coins_won}."
                    prior_messages += result_message + "\n"

                    print("\n" + result_message)

                    print("––––––––––––––––––––––––––\n")
            else:
                messages_without_choice += 1

                if messages_without_choice >= 5:
                    reminder = "Игрок, не забудьте выбрать ручку. Для выбора используйте фразу: 'Крупье, я сделал выбор. Я выбираю ручку N', где N - номер от 1 до 3. Какую ручку вы выберете дальше? Вы обязаны выбрать ручку.'"

                    print(f"Dealer: {reminder}")

                    prior_messages += f"Dealer: {reminder}\n"
                    messages_without_choice = 0

        if rounds >= limit_rounds or coins <= 0:
            net_gain = coins + coins_won - starting_coins
            print(f"\nИгра окончена после {limit_rounds} раундов. Всего монет выиграно: {coins_won}.")
            print(f"Общее количество монет: {coins + coins_won}. Чистый выигрыш (убыток): {net_gain} монет(ы).")
            break

    experiment_number = get_experiment_counter() + 1
    create_pdf_report(Dealer.description, Player.description, full_dialogue, history, win_probabilities, experiment_number)

In [ ]:
for i in range (1):
    if __name__ == "__main__":
        simulate_conversation(limit_rounds=30,
                              shuffle_probabilities=True,
                              win_probabilities=[0.1, 0.4, 1],
                              cheat_probability=0.2,
                              starting_coins=5,
                              cost_per_question=1)

In [ ]:
df_history = pd.DataFrame(history)

df_history['reward'] = np.where(df_history['result'] == 'поражение', 0, 1)
df_history['arm'] = df_history['arm'] + 1

plt.figure(figsize=(10, 6))
plt.plot(df_history['iteration'], df_history['total'], marker='o')
plt.title('Заработанные монеты по итерациям')
plt.xlabel('Итерация')
plt.ylabel('Общая награда')
plt.grid(True)
plt.show()